# 处理附件1中的数据

In [2]:
import pandas as pd
import numpy as np
import os
from pprint import pprint
from scipy import interpolate

# 1 加载数据

In [3]:

data_1_hour_predict_raw = pd.read_excel('../data/附件1 监测点A空气质量预报基础数据.xlsx',sheet_name='监测点A逐小时污染物浓度与气象一次预报数据')
data_1_hour_actual_raw = pd.read_excel('../data/附件1 监测点A空气质量预报基础数据.xlsx',sheet_name='监测点A逐小时污染物浓度与气象实测数据')
data_1_day_actual_raw = pd.read_excel('../data/附件1 监测点A空气质量预报基础数据.xlsx',sheet_name='监测点A逐日污染物浓度实测数据')


In [4]:
print(data_1_hour_predict_raw.head())
print(data_1_hour_actual_raw.head())
print(data_1_day_actual_raw.head())

      模型运行日期                预测时间    地点  近地2米温度（℃）  地表温度（K）  比湿（kg/kg）  \
0 2020-07-23 2020-07-23 00:00:00  监测点A    29.8890  304.016   0.018870   
1 2020-07-23 2020-07-23 01:00:00  监测点A    29.8736  303.739   0.017556   
2 2020-07-23 2020-07-23 02:00:00  监测点A    29.6471  303.419   0.017874   
3 2020-07-23 2020-07-23 03:00:00  监测点A    29.4555  303.419   0.018935   
4 2020-07-23 2020-07-23 04:00:00  监测点A    28.5189  302.987   0.019881   

     湿度（%）  近地10米风速（m/s）  近地10米风向（°）    雨量（mm）  ...  潜热通量（W/m²）  长波辐射（W/m²）  \
0  66.7409       4.16382     162.577  0.000000  ...     0.94818     428.278   
1  62.1551       4.65267     171.978  0.000000  ...     1.14987     427.531   
2  64.1760       4.10031     172.013  0.000000  ...     1.01616     427.428   
3  68.7958       2.44317     168.135  0.047224  ...     1.89003     442.472   
4  76.5791       2.57759     207.884  8.260020  ...     6.53753     458.394   

   短波辐射（W/m²）  地面太阳能辐射（W/m²）  SO2小时平均浓度(μg/m³)  NO2小时平均浓度(μg/m³)  \
0         0.0     

In [5]:
df_1_predict = data_1_hour_actual_raw
df_1_actual = data_1_day_actual_raw
# 更改dataframe的行标签或列标签 https://vimsky.com/examples/usage/python-pandas.DataFrame.set_axis.html
df_1_predict.set_axis(['time', 'place', 'so2','no2','pm10', 'pm2.5', 'o3','co','temperature', 'humidity', 'pressure','wind','direction'], axis='columns', inplace=True)
df_1_actual.set_axis(['time', 'place', 'so2','no2','pm10', 'pm2.5', 'o3','co'], axis='columns', inplace=True)

modeltime_df_actual = df_1_actual['time']
modeltime_df_pre = df_1_predict['time']

df_1_actual = df_1_actual.drop(columns=['place','time'])
df_1_predict = df_1_predict.drop(columns=['place','time'])
df_1_predict = df_1_predict.replace('—', np.nan)
df_1_predict = df_1_predict.astype('float')
df_1_predict[ df_1_predict< 0 ] = np.nan
# 重新插入time列
df_1_actual.insert(0, 'time', modeltime_df_actual)
df_1_predict.insert(0, 'time', modeltime_df_pre)

# 线性插值的方法需要单独处理最后一行的数据
data_1_actual = df_1_actual[0:-3]

data_1_predict = df_1_predict
data_1_predict.iloc[-1:]['pm10'] =22.0

data_1_actual_knn = df_1_actual[0:-3]
data_1_predict_knn =df_1_predict

D:\Temp\ipykernel_27164\1848872862.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1_predict.iloc[-1:]['pm10'] =22.0


# 2 插值

## 2.1 线性插值

In [8]:

for indexs in data_1_actual.columns:
    if indexs =='time':
        continue
    data_1_actual['rownum'] = np.arange(data_1_actual.shape[0])
    df_nona = data_1_actual.dropna(subset = [indexs])#当前位置元素为空，删除当前元素所在的行
    f = interpolate.interp1d(df_nona['rownum'], df_nona[indexs]) #构建从 rownum 到 具体字段值 的函数映射
    data_1_actual[indexs] = f(data_1_actual['rownum'])

data_1_actual = data_1_actual.drop(columns=['rownum'])

for indexs in data_1_predict.columns:
    if indexs =='time':
        continue
    data_1_predict['rownum'] = np.arange(data_1_predict.shape[0])
    df_nona = data_1_predict.dropna(subset = [indexs])
    f = interpolate.interp1d(df_nona['rownum'], df_nona[indexs],fill_value="extrapolate")
    data_1_predict[indexs] = f(data_1_predict['rownum'])
data_1_predict = data_1_predict.drop(columns=['rownum'])


In [10]:
writer=pd.ExcelWriter('../data/data_1_linear.xlsx')   #定义writer
data_1_hour_predict_raw.to_excel(writer,'监测点A逐小时污染物浓度与气象一次预报数据',index=False) 
data_1_predict.to_excel(writer,'监测点A逐小时污染物浓度与气象实测数据',index=False) # writer---writer；sheet名称---原始表（35698）
data_1_actual.to_excel(writer,'监测点A逐日污染物浓度实测数据',index=False)

writer.save()

In [11]:
data_1_predict

,time,so2,no2,pm10,pm2.5,o3,co,temperature,humidity,pressure,wind,direction
0,2019-04-16 00:00:00,5.0,75.0,61.0,38.0,1.0,1.0,21.1,89.0,1013.0,1.2,346.5
1,2019-04-16 01:00:00,5.0,64.0,60.0,41.0,25.0,0.8,20.4,92.0,1012.4,1.1,18.0
2,2019-04-16 02:00:00,4.0,60.0,36.0,40.0,25.0,0.8,20.0,93.0,1011.3,1.1,65.9
3,2019-04-16 03:00:00,4.0,39.0,35.0,33.0,41.0,0.7,19.8,93.0,1010.4,1.5,78.4
4,2019-04-16 04:00:00,4.0,39.0,28.0,39.0,36.0,0.8,19.9,92.0,1010.0,1.5,73.5
...,...,...,...,...,...,...,...,...,...,...,...,...
19427,2021-07-13 03:00:00,6.0,12.0,12.0,0.0,14.0,0.4,29.2,78.0,1006.2,0.8,350.5
19428,2021-07-13 04:00:00,7.0,15.0,9.0,0.0,11.0,0.4,29.2,78.0,1006.3,0.7,246.3
19429,2021-07-13 05:00:00,9.0,19.0,9.0,2.0,8.0,0.4,29.2,78.0,1006.4,1.0,253.2
19430,2021-07-13 06:00:00,8.0,20.0,22.0,5.0,6.0,0.4,29.0,81.0,1006.8,0.5,267.9


## 2.2 KNN插值

In [12]:

def knn_mean(ts, n):
    out = np.copy(ts)
    for i, val in enumerate(ts):
        if np.isnan(val):
            n_by_2 = np.ceil(n/2)
            lower = np.max([0, int(i-n_by_2)])
            upper = np.min([len(ts)+1, int(i+n_by_2)])
            ts_near = np.concatenate([ts[lower:i], ts[i:upper]])
            out[i] = np.nanmean(ts_near)
    return out
for indexs in data_1_actual_knn.columns:
    if indexs =='time':
        continue
    data_1_actual_knn[indexs] = knn_mean(data_1_actual_knn[indexs].values,8)
for indexs in data_1_predict_knn.columns:
    if indexs =='time':
        continue
    data_1_predict_knn[indexs] = knn_mean(data_1_predict_knn[indexs].values,8)

D:\Temp\ipykernel_27164\3492667378.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1_actual_knn[indexs] = knn_mean(data_1_actual_knn[indexs].values,8)


In [19]:
data_1_predict_knn
# data_1_predict_knn_time=data_1_predict_knn['time']
# data_1_predict_knn=data_1_predict_knn.drop(columns=['time'])
print(data_1_predict_knn.head())

   so2   no2  pm10  pm2.5    o3   co  temperature  humidity  pressure  wind  \
0  5.0  75.0  61.0   38.0   1.0  1.0         21.1      89.0    1013.0   1.2   
1  5.0  64.0  60.0   41.0  25.0  0.8         20.4      92.0    1012.4   1.1   
2  4.0  60.0  36.0   40.0  25.0  0.8         20.0      93.0    1011.3   1.1   
3  4.0  39.0  35.0   33.0  41.0  0.7         19.8      93.0    1010.4   1.5   
4  4.0  39.0  28.0   39.0  36.0  0.8         19.9      92.0    1010.0   1.5   

   direction  rownum  
0      346.5     0.0  
1       18.0     1.0  
2       65.9     2.0  
3       78.4     3.0  
4       73.5     4.0  


In [26]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=10)
res=imputer.fit_transform(data_1_predict_knn).astype('float')
# data_1_predict_knn=data_1_predict_knn.drop(columns=['rownum'])


#numpy.ndarray 转为 pandas.dataframe
res=pd.DataFrame(res, columns=['so2','no2','pm10', 'pm2.5', 'o3','co','temperature', 'humidity', 'pressure','wind','direction'])
print(res)
writer=pd.ExcelWriter('../data/data_test.xlsx')   #定义writer
res.to_excel(writer,'test',index=False) 

writer.save()

       so2   no2  pm10  pm2.5    o3   co  temperature  humidity  pressure  \
0      5.0  75.0  61.0   38.0   1.0  1.0         21.1      89.0    1013.0   
1      5.0  64.0  60.0   41.0  25.0  0.8         20.4      92.0    1012.4   
2      4.0  60.0  36.0   40.0  25.0  0.8         20.0      93.0    1011.3   
3      4.0  39.0  35.0   33.0  41.0  0.7         19.8      93.0    1010.4   
4      4.0  39.0  28.0   39.0  36.0  0.8         19.9      92.0    1010.0   
...    ...   ...   ...    ...   ...  ...          ...       ...       ...   
19427  6.0  12.0  12.0    0.0  14.0  0.4         29.2      78.0    1006.2   
19428  7.0  15.0   9.0    0.0  11.0  0.4         29.2      78.0    1006.3   
19429  9.0  19.0   9.0    2.0   8.0  0.4         29.2      78.0    1006.4   
19430  8.0  20.0  22.0    5.0   6.0  0.4         29.0      81.0    1006.8   
19431  8.0  14.0  35.0    5.0  19.0  0.4         30.2      75.0    1007.0   

       wind  direction  
0       1.2      346.5  
1       1.1       18.0  


In [ ]:
writer=pd.ExcelWriter('data/data_1_knn.xlsx')   #定义writer

data_1_hour_predict_raw.to_excel(writer,'监测点A逐小时污染物浓度与气象一次预报数据',index=False) 
data_1_predict_knn.to_excel(writer,'监测点A逐小时污染物浓度与气象实测数据',index=False) # writer---writer；sheet名称---原始表（35698）
data_1_actual_knn.to_excel(writer,'监测点A逐日污染物浓度实测数据',index=False)

writer.save()

: 

In [27]:
data_1_predict_knn

,so2,no2,pm10,pm2.5,o3,co,temperature,humidity,pressure,wind,direction
0,5.0,75.0,61.0,38.0,1.0,1.0,21.1,89.0,1013.0,1.2,346.5
1,5.0,64.0,60.0,41.0,25.0,0.8,20.4,92.0,1012.4,1.1,18.0
2,4.0,60.0,36.0,40.0,25.0,0.8,20.0,93.0,1011.3,1.1,65.9
3,4.0,39.0,35.0,33.0,41.0,0.7,19.8,93.0,1010.4,1.5,78.4
4,4.0,39.0,28.0,39.0,36.0,0.8,19.9,92.0,1010.0,1.5,73.5
...,...,...,...,...,...,...,...,...,...,...,...
19427,6.0,12.0,12.0,0.0,14.0,0.4,29.2,78.0,1006.2,0.8,350.5
19428,7.0,15.0,9.0,0.0,11.0,0.4,29.2,78.0,1006.3,0.7,246.3
19429,9.0,19.0,9.0,2.0,8.0,0.4,29.2,78.0,1006.4,1.0,253.2
19430,8.0,20.0,22.0,5.0,6.0,0.4,29.0,81.0,1006.8,0.5,267.9
